In [1]:
import argparse
import os
import shutil
import gzip

import torch
import torch.nn as nn
from torch.autograd import Variable

import torch
import torch.nn as nn
from torch.autograd import Variable

from dpp_nets.utils.language import Vocabulary, BeerDataset, custom_collate
from dpp_nets.layers.baselines import NetBaseline, SetNetBaseline, AttentionBaseline

from dpp_nets.utils.language import EvalSet

In [2]:
# Load saved checkpoint
model = 'shortsentsNet1lr0.01best_ckp.pth.tar'
model_dir = '/Users/Max/checkpoints/beer_reviews/baseline/' 
model_path = model_dir + model
model = torch.load(model_path, map_location=lambda storage, loc: storage)

In [3]:
from dpp_nets.utils.language import Vocabulary

embd_path = '/Users/Max/data/beer_reviews/review+wiki.filtered.200.txt.gz'
word_path = '/Users/Max/data/beer_reviews/reviews.all.train.words.txt.gz'

# Set-up Vocabulary
vocab = Vocabulary()
vocab.loadPretrained(embd_path)
vocab.setStops()
vocab.loadCorpus(word_path)
vocab.updateEmbedding()
vocab.setCuda(False)
vocab.EmbeddingBag.load_state_dict(model['embedding'])

In [4]:
EMBD_DIM = 200
KERNEL_DIM = 200
HIDDEN_DIM = 500
ENC_DIM = 200
TARGET_DIM = 3 if model['aspect'] in set(['all', 'short']) else 1

if model['model_type'] == NetBaseline:
    trainer = NetBaseline(EMBD_DIM, HIDDEN_DIM, TARGET_DIM)
if model['model_type'] == SetNetBaseline:
    trainer = NetBaseline(EMBD_DIM, HIDDEN_DIM, KERNEL_DIM, ENC_DIM, TARGET_DIM)
if model['model_type'] == AttentionBaseline:
    trainer = SetNetBaseline(EMBD_DIM, HIDDEN_DIM, ENC_DIM, TARGET_DIM)

trainer.load_state_dict(model['model'])
#trainer.activation = nn.Sigmoid()

rat_path = '/Users/Max/data/beer_reviews/annotations.json'
evalset = EvalSet(rat_path, vocab)

In [5]:
evalset.evaluateBaseline(trainer, model['mode'])

0.01592683966787016

In [ ]:
print('__________________________Training Table__________________________')
for k, v in model['train_loss'].items():
    print(str.join(" | ", ['Epoch: %d' % (k), 'Loss: %.5f' % (v)]))

In [ ]:
from dpp_nets.helper.plotting import plot_floats

# Training Plots
plot_floats(model['train_loss'], xlabel='Epochs', ylabel='MSE + Reg', title='Training MSE + Reg')

In [ ]:
print('_________________________Validation Table_________________________')
for k, v in model['val_loss'].items():
    print(str.join(" | ", ['Epoch: %d' % (k), 'Loss: %.5f' % (v)]))

In [ ]:
from dpp_nets.helper.plotting import plot_floats

# Training Plots
plot_floats(model['val_loss'], xlabel='Epochs', ylabel='MSE + Reg', title='Validation MSE + Reg')

In [ ]:
# Evaluation on Test Set

loss, pred_loss, reg_loss = evalset.computeLoss(trainer, model['mode'])
print(str.join(" | ", ['Test Set:', 'Loss: %.5f' % (loss), 
                              'Pred Loss: %.5f' % (pred_loss), 'Reg Loss: %.5f' % (reg_loss)]))

prec, extract = evalset.evaluatePrecision(trainer,model['mode'])
print(str.join(" | ", ['Test Set:', 'Precision: %.5f' % (prec), 'Extract: %.5f' % (extract)]))

In [ ]:
# Random Samples

evalset.sample(trainer, model['mode'])

In [ ]:
# Random Marginals
evalset.computeMarginals(trainer, model['mode'])